In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [4]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [10]:
df = pd.read_csv("C:\\Users\\riffere\\Desktop\\zane_zips.csv")
df

,zip_code,Owner Postal Code,Owner City,Total Price,Total Sale Quantity
0,00726,00726,Caguas,522.16,2
1,00917,00917,San Juan,246.78,3
2,00926,00926,San Juan,807.64,2
3,01105,01105,Springfield,329.04,4
4,01376,01376,Ciudad de México,261.08,1
...,...,...,...,...,...
545,L7A,L7A3R8,Brampton,383.76,4
546,M3M,M3M2W4,Toronto,246.60,2
547,M9B,M9B5G1,Toronto,653.92,2
548,P1P,P1P1R2,Gravenhurst,164.52,2


In [17]:
q = """
select * from custom.zipcodes_aggregate
"""
df2 = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)
df2

,zip_code,cities,counties,states,country,processed_date
0,A0E,"Baine Harbour, Bay l'Argent, Boat Harbour West...","Burin Peninsula,Clarenville",NL,CA,2023-03-10 12:11:34.209032
1,A0K,"Anchor Point, Baie Verte, Bartletts Harbour, B...","Clarenville,Corner Brook,Labrador City,Wiltond...",NL,CA,2023-03-10 12:11:34.209032
2,A1K,"Bauline, Flatrock, Logy Bay, Middle Cove, Oute...",Conception Bay - St. Johns,NL,CA,2023-03-10 12:11:34.209032
3,A1X,Conception Bay South,Conception Bay - St. Johns,NL,CA,2023-03-10 12:11:34.209032
4,A8A,"Bonne Bay Pond, Cormack, Deer Lake, Pynn's Bro...","Corner Brook,Wiltondale",NL,CA,2023-03-10 12:11:34.209032
...,...,...,...,...,...,...
40651,99811,Sterling,Kenai Peninsula Borough,AK,US,2023-03-10 12:11:34.209032
40652,99827,Elfin cove,Hoonah-Angoon Census Area,AK,US,2023-03-10 12:11:34.209032
40653,99835,Skagway,Skagway Municipality,AK,US,2023-03-10 12:11:34.209032
40654,99923,Coffman cove,Prince of Wales-Hyder Census Area,AK,US,2023-03-10 12:11:34.209032


In [18]:
df = df.merge(df2, on = "zip_code", how = 'left')

In [19]:
df.to_csv('C:\\Users\\riffere\\Desktop\\zipx_final1.csv')